In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install geopandas
!pip install pyshp

import os 
import pickle
import zipfile
import pyproj

import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString

import shapefile
from pyproj import CRS, Transformer

import pathlib
from glob import glob
import glob
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import warnings
warnings.filterwarnings('ignore')

path = Path('/content/drive/MyDrive/Colab Notebooks/PineTree')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 6.3 MB 52.3 MB/s 
     |████████████████████████████████| 16.7 MB 48.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.3 MB/s 


In [ ]:
with open(str(path / 'weather' / 'asos.pickle'), 'rb') as f:
  asos = pickle.load(f)

with open(str(path / 'weather' / 'aws.pickle'), 'rb') as f:
  aws = pickle.load(f)


## utilize_file.py

In [ ]:
def unzip_bugoccur(path, second_folder_name):
  """
  path: Path type, home(PineTree)
  """
  zip_list = glob(str(path / second_folder_name / '**/**.zip'))

  zip_list
  for z in zip_list:
    p = z.split('/')
    new_path = path / p[-3] / p[-2] / p[-1].split('_')[-1][:-4]
    os.mkdir(new_path)
    zipfile.ZipFile(z).extractall(str(new_path))

def file_rename(path, second_folder_name, rename_type):
  """
  to rename file for efficient processing
  rename_type: start '_' end '.'  => ex) _bugtree.
  """

  paths = [x for x  in glob.iglob(str(path / second_folder_name / '*' ),recursive=True) if os.path.isdir(x)]

  shp_path = []
  for p in paths:
    gb = glob.glob(p + '/*.*')
    if gb != []:

      if gb[0].split('.')[0].split('_')[-1] == rename_type[1:-1]:
        print("The name has already been changed.")
        break
      for g in gb:
        sg = g.split('/')
        os.rename(g, '/'.join(sg[:-1]) + '/' + sg[-1].split('.')[0] + rename_type+ sg[-1].split('.')[-1])

## put_localcode.py

In [16]:
def extract_type(path, second_folder_name, extracted_type='shp'):
  paths = [x for x  in glob.iglob(str(path / second_folder_name / '*'),recursive=True) if os.path.isdir(x)]

  shp_path = []
  for p in paths:
    gb = glob.glob(p + '/*.'+extracted_type)
    if gb != []:
      shp_path.append(*gb)
  
  return shp_path

# def input_lcd(path, second_folder_name, shp_path, storage_num=7):
#   """
#   to put local code in the shp file format
#   storage_num: local_code폴더가 위치한 넘버
#   """

#   fst_local = shp_path[0].split('/')[storage_num]
#   fst_act = True

#   df_1 = gpd.GeoDataFrame()

#   for idx, sh in enumerate(shp_path):      
#     shc = sh.split('/')
#     if os.path.isdir(str(path / second_folder_name / shc[storage_num])):
#       print(f'pass: {shc[storage_num]}')
#       continue

#     else:
#       if fst_act:
#         fst_local = shp_path[idx].split('/')[storage_num]
#         fst_act = False

#       print(f'idx: {idx}\tfst_local: {fst_local}')

#     if fst_local != shc[storage_num]:
#       print(f'save: {len(df_1)}')
#       df_1.to_file(str(path / second_folder_name / fst_local), driver='ESRI Shapefile')
      
#       fst_local = shc[storage_num]
#       fst_act = True
#       df_1 = gpd.GeoDataFrame()

#     lcd = int(shc[-1].split('.')[0].split('_')[-1])
#     df_2 = gpd.read_file(sh)
#     df_2['LOCAL_CD'] = lcd
    
#     df_1 = gpd.GeoDataFrame( pd.concat( [df_1, df_2], ignore_index=True) )

#   df_1 = gpd.GeoDataFrame( pd.concat( [df_1, df_2], ignore_index=True) )
#   print(f'save: {len(df_1)}')
#   df_1.to_file(str(path / second_folder_name / fst_local), driver='ESRI Shapefile')


def input_lcd(path, second_folder_name, shp_path, storage_num=7):
  """
  to put local code in the shp file format
  storage_num: local_code폴더가 위치한 넘버
  """

  fst_local = shp_path[0].split('/')[storage_num]
  fst_act = True

  df_1 = gpd.GeoDataFrame()

  for idx, sh in enumerate(shp_path):      
    shc = sh.split('/')

    lcd = int(shc[-1].split('.')[0].split('_')[-1])
    df_2 = gpd.read_file(sh)
    df_2['LOCAL_CD'] = lcd
    
    df_1 = gpd.GeoDataFrame( pd.concat( [df_1, df_2], ignore_index=True) )

  df_1 = gpd.GeoDataFrame( pd.concat( [df_1, df_2], ignore_index=True) )
  print(f'save: {len(df_1)}')
  df_1.to_file(str(path / second_folder_name / fst_local), driver='ESRI Shapefile')

In [20]:
shp_path = extract_type(path, '경기도')

In [26]:
fst_local = shp_path[0].split('/')[7]

fst_act = True

df_1 = gpd.GeoDataFrame()

for idx, sh in enumerate(shp_path):      
  shc = sh.split('/')

  lcd = int(shc[-1].split('.')[0].split('_')[-1])
  print(lcd)

  df_2 = gpd.read_file(sh)
  df_2['LOCAL_CD'] = lcd
  
  df_1 = gpd.GeoDataFrame( pd.concat( [df_1, df_2], ignore_index=True) )

df_1 = gpd.GeoDataFrame( pd.concat( [df_1, df_2], ignore_index=True) )
print(f'save: {len(df_1)}')
df_1.to_file(str(path / '경기도' / 'gyeonggi.csv'), driver='ESRI Shapefile')

41390
41820
41370
41610
41173
41171
41360
41630
41463
41271
41590
41220
41550
41450
41480
41465
41461
41150
41650
41430
41830
41670
41410
41287
41210
41800
41310
41500
41281
41285
41111
41135
41117
41113
41115
41133
41250
41131
41570
41290
41273
save: 342596


In [24]:
fst_local

'FRT001003_41390'

In [23]:
shc = shp_path[0].split('/')
lcd = int(shc[-1].split('.')[0].split('_')[-1])
lcd

41390

In [28]:
sejong = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/PineTree/pinetrees/sejong/sejong_bugtree.shp')
sejong

,OCCRR_NO,DBHIS_CD,MNAGN_CD,ARA_XCRD,ARA_YCRD,NTN_SPOT,LGDNG_CD,IFTTR_TPCD,EXMNN_OTST,FRST_RGTER,FRST_RGSTN,OCCRR_SEQ,LAST_UPUSR,LAST_UPDT,LOCAL_CD,geometry
0,12603650.0,6380.0,5690000,232834,435495,??88053550,3611033021,None,20180510,None,None,1.0,None,None,36110,POINT (988053.057 1835506.607)
1,12603651.0,6380.0,5690000,232835,435518,??88053552,3611033021,None,20180510,None,None,2.0,None,None,36110,POINT (988054.176 1835529.592)
2,12603652.0,6380.0,5690000,232837,435499,??88053551,3611033021,None,20180510,None,None,3.0,None,None,36110,POINT (988056.077 1835510.590)
3,5058588.0,849.0,1400467,220146,453617,??75465368,3611038029,02,20170210,None,None,2.0,None,None,36110,POINT (975464.611 1853687.019)
4,5058589.0,849.0,1400467,220149,453607,??75465367,3611038029,02,20170210,None,None,3.0,None,None,36110,POINT (975467.558 1853677.008)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16024,17649231.0,1001.0,5690000,218009,449028,??733049,3611037032,None,20200604,None,None,2.0,None,None,36110,POINT (973304.586 1849111.027)
16025,17649232.0,1001.0,5690000,217986,449024,??732849,3611037032,None,20200604,None,None,3.0,None,None,36110,POINT (973281.574 1849107.148)
16026,17649233.0,1001.0,5690000,218004,449011,??732949,3611037032,None,20200604,None,None,4.0,None,None,36110,POINT (973299.499 1849094.060)
16027,17649234.0,1001.0,5690000,218012,449031,??733049,3611037032,None,20200604,None,None,5.0,None,None,36110,POINT (973307.600 1849114.010)


In [30]:
sejong.columns

Index(['OCCRR_NO', 'DBHIS_CD', 'MNAGN_CD', 'ARA_XCRD', 'ARA_YCRD', 'NTN_SPOT',
       'LGDNG_CD', 'IFTTR_TPCD', 'EXMNN_OTST', 'FRST_RGTER', 'FRST_RGSTN',
       'OCCRR_SEQ', 'LAST_UPUSR', 'LAST_UPDT', 'LOCAL_CD', 'geometry'],
      dtype='object')

In [29]:
with open ('/content/drive/MyDrive/Colab Notebooks/PineTree/bug_re/busan_bug', 'rb') as f:
  busan = pickle.load(f)

busan

,SEED,OCCRR_NO,DBHIS_CD,MNAGN_CD,ARA_XCRD,ARA_YCRD,EXMNN_OTST,LOCAL_CD,STORUNST,FROR_CD,...,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,LDMARK_STN,MAP_LABEL,LATITUDE,LONGITUDE,STATION_ID
0,0,13154495,6875,3260000,383490.0,279725.0,20180702,26140,1,1,...,17,1,3,C,12,J11117,ACP-13C,1.137837e+06,1.679047e+06,159
1,1,13154496,6875,3260000,383507.0,279733.0,20180702,26140,1,1,...,17,1,3,C,12,J11117,ACP-13C,1.137854e+06,1.679055e+06,159
2,2,13154497,6875,3260000,384291.0,278824.0,20180702,26140,1,2,...,15,2,3,C,12,J12115,NPT-23C,1.138633e+06,1.678142e+06,159
3,3,13154498,6875,3260000,384280.0,278771.0,20180702,26140,1,2,...,15,2,3,C,12,J12115,NPT-23C,1.138622e+06,1.678089e+06,159
4,4,13154500,6875,3260000,383810.0,276600.0,20180702,26140,1,2,...,15,2,4,C,14,J12115,NPT-24C,1.138141e+06,1.675922e+06,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131326,131326,17648498,1001,3400000,398210.0,306318.0,20200527,26710,1,2,...,15,2,4,C,16,J12115,NPT-24C,1.152682e+06,1.705550e+06,257
131327,131327,17648499,1001,3400000,398231.0,306353.0,20200527,26710,1,2,...,15,2,4,C,16,J12115,NPT-24C,1.152704e+06,1.705585e+06,257
131328,131328,17648500,1001,3400000,398258.0,306337.0,20200527,26710,1,2,...,15,2,4,C,16,J12115,NPT-24C,1.152730e+06,1.705568e+06,257
131329,131329,17746609,1001,5380000,394538.0,303270.0,20201123,26710,1,2,...,34,2,5,C,16,J12234,NQQ-25C,1.148997e+06,1.702522e+06,257


In [31]:
busan.columns

Index(['SEED', 'OCCRR_NO', 'DBHIS_CD', 'MNAGN_CD', 'ARA_XCRD', 'ARA_YCRD',
       'EXMNN_OTST', 'LOCAL_CD', 'STORUNST', 'FROR_CD', 'FRTP_CD',
       'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'DNST_CD', 'HEIGHT', 'LDMARK_STN',
       'MAP_LABEL', 'LATITUDE', 'LONGITUDE', 'STATION_ID'],
      dtype='object')

## upload_path.py

In [ ]:
# path
path  = Path('/content/drive/MyDrive/Colab Notebooks/PineTree')
os.chdir(str(path))

# 임상도 지역코드
df_fcd = pd.read_csv(str(path / 'LocalCode' / 'Kangwon' / 'FloorPlan.csv'), encoding='cp949')

# Variable_path
bug_path_list = [] # 소나무 재선충 발생 파일 리스트
bug_path = path / 'BugOccur' / 'Kangwon'

for p in bug_path.iterdir():
  if str(p).split('/')[-1].split('.')[0].isdigit() and str(p).split('/')[-1].split('.')[1] == 'csv':
    bug_path_list.append(p)

floor_path_list = [] # 소나무류 임상도 파일 경로 리스트
floor_path = path / 'FloorPlan' / 'LocalFloorPlan' / 'Kangwon'

for p in floor_path.iterdir():
  if p.is_dir() and str(p).split('/')[-1] != '.ipynb_checkpoints':
    floor_path_list.append(glob(str(p) + '/**.shp')[0])
    print(p)

## make_dataframe.py

In [27]:
# DataFrame

def make_bug(bug_path_list):
  """
  To make dataframe about pine reworm
  """
  df_bug = pd.DataFrame() # 소나무 재선충 발생 데이터 프레임

  for p in bug_path_list:
    tmp = pd.read_csv(p, encoding='cp949')
    tmp['LOCAL_CODE'] = int(str(p).split('/')[-1][:-4])
    df_bug = pd.concat([df_bug, tmp])

  df_bug = df_bug.reset_index(drop=True)
  df_bug = gpd.GeoDataFrame(df_bug)
  
  return df_bug


def make_floor(floor_path_list):
  """
  To make dataframe about pinetree floor map
  """

  df_floor = gpd.GeoDataFrame() # 소나무류 임상도 데이터 프레임

  for p in floor_path_list:
    tmp = gpd.GeoDataFrame.from_file(str(p), encoding='cp949')
    tmp['LOCAL_CODE'] = int(str(p).split('/')[-1][:-4].split('_')[-1])

    df_floor = gpd.GeoDataFrame( pd.concat([df_floor, tmp], ignore_index=True))

  return df_floor


def make_weather(local_path='Kangwon'):
  """
  To make dataframe about the day of the nearest observatory at that point
  df_asos: 기상관측정보(전국)
  df_aws: 방재기상관측정보(전국)
  """
  df_asos = pd.read_csv(str(path / 'LocalWeather' / local_path / 'OBS_ASOS_MNH.csv'), encoding='cp949')
  df_aws = pd.read_csv(str(path / 'LocalWeather' / local_path / 'OBS_AWS_MNH.csv'), encoding='cp949')

  # missing value
  aws_supple_list = glob(str(path / 'LocalWeather' / local_path.capitalize() / 'SupplementalDate' / '**.csv'))
  aws_supple_list.sort()

  for idx in range(len(aws_supple_list)):
    n = aws_supple_list[idx].split('/')[-1].split('_')
    m = n[-1].split('.')

    exec(f'{n[1].lower()}_{n[2]}_{m[0]} = pd.read_csv(aws_supple_list[{idx}], encoding="cp949")')

  table_define = {'4': ['평균기온(°C)', 1], 
                '5': ['평균최고기온(°C)', 4], '6': ['평균최저기온(°C)', 2], '11': ['평균풍속(m/s)', 9]}

  # table_define = {'4': ['평균기온(°C)', 1], '5': ['평균최고기온(°C)', 4], '6': ['평균최저기온(°C)', 2],
  #                 '7': ['최고기온(°C)', 4], '8': ['최저기온(°C)', 2], '11': ['평균풍속(m/s)', 9],
  #                 '12': ['최대순간풍속(m/s)', 8], '13': ['최대순간풍속풍향(deg)', 10], '14': ['월강수량합(mm)', 6]}

  for i in df_aws[df_aws['평균풍속(m/s)'].isnull()].sort_values(by=['일시'], axis=0, ascending=True, inplace=False).itertuples():
    table = i.지점명
    time = i.일시
    index = i.Index

    table_ = []
    for idx in range(16):
      if pd.isna(i[idx]):
        table_.append(idx)

    k = time.split('-')
    exec(f't = aws_{k[0]}_{k[1]}[aws_{k[0]}_{k[1]}.지점명 == "{table}"].describe().mean()')

    for z in table_:
      if str(z) in table_define.keys():
        exec(f'aws.loc[{index},"{table_define[str(z)][0]}"] = t[{table_define[str(z)][1]}]')



## modify_dataframe.py

In [ ]:
def transform_name(df, columns, new_columns, drop=True):
  """
  Switch to the name of the database type
  drop: True이면, columns 삭제, False: columns: 선택
  """
  df = df.drop(columns, axis=1)
  
  if len(df.columns) == len(new_columns):
    df.columns = new_columns
    return df

  else:
    raise ValueError('Incorrect number of columns to convert')


def extract_local_ws(df_weather_station, sido):
  """
  Extract only observatories for local weather
  sido: main sido first, sub sido second
  """
  idx_ = []

  for idx, row in enumerate(df_weather_station.itertuples()):
  
    if type(sido) == list:
      if (row.station_do == sido[0]) or (row.station_do == sido[1]):
        idx_.append(idx)

    else:
      if (row.station_do == sido):
        idx_.append(idx)

  df_ws = df_weather_station.loc[idx_]
  df_ws = df_ws.reset_index()
  df_ws = df_ws.drop(['index'], axis=1)

  if type(sido) == list:
    df_ws['station_do'] = df_ws['station_do'].apply(lambda x: sido[0]  if x == sido[1] else sido[0])

  df_ws = df_ws.drop(['station_do', 'station_si', 'station_gungu', 'station_dong', 'station_eup', 'station_myeon', 'station_ri'], axis=1)

  df_ws['latitude'] = -1
  df_ws['longitude'] = -1

  return df_ws

def apply_coordinates(df):
  df_station = pd.read_csv(str(path / 'LocalCode' / 'station.csv'))

  no_lat_mapping = []
  no_long_mapping = []


  for row in df.itertuples(): 

    condition = (df_station.지점 == row[1]) & (df_station.지점명 == row[2])

    lat = list(df_station[condition]['STATION_LATITUDE'])
    log = list(df_station[condition]['STATION_LONGITUDE'])

    df.loc[row[0], 'latitude'] =  lat[0] if len(lat) != 0 else no_lat_mapping.append(row)
    df.loc[row[0], 'longitude'] = log[0] if len(log) != 0 else no_long_mapping.append(row)

  null_lat_mapping = []
  null_long_mapping = []

  for row in no_lat_mapping:
    condition = (df_station.지점 == row[1])

    lat = list(df_station[condition]['위도'])
    log = list(df_station[condition]['경도'])

    df.loc[row[0], 'latitude'] =  lat[0] if len(lat) != 0 else null_lat_mapping.append(row)
    df.loc[row[0], 'longitude'] = log[0] if len(log) != 0 else null_long_mapping.append(row)

  df.columns = ['STATION_ID', 'STATION_NAME', 'STATION_LATITUDE', 'STATION_LONGITUDE']
  df.STATION_LATITUDE = df.STATION_LATITUDE.astype('float')
  df.STATION_LONGITUDE = df.STATION_LONGITUDE.astype('float')
  
  return df

def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 4], coord[:, 3])
    return np.dstack([fx, fy])[0]


## map_weather.py

In [ ]:
def average_weather_data(file_name):
  asos_np, aws_np = [], []
  for station, a_np in zip([asos_dd, aws_dd], [asos_np, aws_np]):

    station_id = list(set(station.STATION_ID))
    station_id.sort()

    for id in station_id:
      print(id)
      a_tmp = []
      for month in range(1, 13):
        condition = (station.STATION_ID == id) & ((station.DATE.dt.strftime('%Y-%m') == f'2019-{month:02}') | \
                                                      (station.DATE.dt.strftime('%Y-%m') == f'2020-{month:02}') | \
                                                      (station.DATE.dt.strftime('%Y-%m') == f'2021-{month:02}') | \
                                                      (station.DATE.dt.strftime('%Y-%m') == f'2022-{month:02}')) 

        tmp = station[condition].groupby('STATION_ID').mean().to_numpy()
        a_tmp.append(tmp)
      a_np.append(a_tmp)

  with open(str(path / 'weather' / file_name), 'wb') as f:
    pickle.dump(asos_np, f)


def make_weather_dataframe():
  aws_np = []
  for row in aws:
    row_fst = row[0]
  
    for idx, r in enumerate(row):
      if idx != 0:
        row_fst = np.concatenate([row_fst, r])
    row_fst = row_fst.reshape(-1)
    aws_np.append(row_fst)

  aws_np = np.array(aws_np)


In [ ]:
def preprocess_weather(dataframe1=asos, dataframe2=aws):

  weather_list = glob.glob(str(path / 'weather' / '*.csv'))
  weather_list.sort()

  asos_dd = pd.read_csv(weather_list[0], encoding='cp949')
  aws_dd = pd.read_csv(weather_list[2], encoding='cp949')

  asos_cols = ['지점', '지점명', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '최대 순간 풍속 풍향(16방위)', 
              '최대 풍속(m/s)', '최대 풍속 풍향(16방위)', '평균 풍속(m/s)', '최다풍향(16방위)', '평균 이슬점온도(°C)', 
              '최소 상대습도(%)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 현지기압(hPa)', '가조시간(hr)',
              '평균 지면온도(°C)', '최저 초상온도(°C)']

  asos_dd = asos_dd[asos_cols]

  asos_dd.columns = ['STATION_ID', 'STATION_NAME', 'DATE', 'CSS_AVG', 'MIN_CSS_AVG', 'MAX_CSS_AVG', 'MAX_IWS_DEG',
                    'MAX_WS_MS', 'MAX_WS_DEG', 'WS_AVG_MS', 'MAX_WO_DEG', 'CSS_DEW_AVG',
                    'MIN_REL_HUMID', 'REL_HUMID_AVG', 'VAPOR_AVG', 'VAPOR_LOCAL_AVG', 'PDOS_HR',
                    'CSS_GRD_AVG', 'MIN_CSS_GRD']

  asos_dd.DATE = pd.to_datetime(asos_dd.DATE)

  aws_cols = ['지점', '지점명', '일시', '평균기온(°C)', 
              '최저기온(°C)', '최고기온(°C)', '일강수량(mm)',
              '최대 순간 풍속(m/s)', '평균 풍속(m/s)', '최대 순간 풍속 풍향(deg)']

  aws_dd = aws_dd[aws_cols]

  aws_dd.columns = ['STATION_ID', 'STATION_NAME', 'DATE', 'CSS_AVG', 
                    'MIN_CSS_AVG', 'MAX_CSS_AVG', 'RF_AVG',
                    'MAX_WS_MS', 'WS_MS', 'MAX_IWS_DEG']

  aws_dd.DATE = pd.to_datetime(aws_dd.DATE)


  aws_np = []
  aws_not_12 = []
  for i, row in enumerate(aws):
    row_fst = row[0]

    for idx, r in enumerate(row):
      if idx != 0:
        row_fst = np.concatenate([row_fst, r])

    row_fst = row_fst.reshape(-1).tolist()

    if len(row_fst) != 84:
      aws_not_12.append(i)
    
    else:
      aws_np.append(row_fst)

  aws_np = np.array(aws_np)

  station_id = list(set(aws_dd.STATION_ID))
  station_id.sort()

  df = pd.DataFrame()

  for idx, id in enumerate(station_id):
    if not idx in aws_not_12:
      df = pd.concat([df, aws_dd[aws_dd.STATION_ID == id].head(1)[['STATION_ID', 'STATION_NAME']]], axis=0)

  df = np.array(df)
  df = np.concatenate([df, aws_np], axis=1)

  aws_columns = ['STATION_ID', 'STATION_NAME']
  [aws_columns.extend([f'CSS_AVG_{month:02}', f'MIN_CSS_AVG_{month:02}', f'MAX_CSS_AVG_{month:02}', f'RF_AVG_{month:02}', f'MAX_WS_MS_{month:02}', f'WS_MS_{month:02}', f'MAX_IWS_DEG_{month:02}']) for month in range(1, 13)]

  aws_df = pd.DataFrame(data=df, columns=aws_columns)

  asos_np = []
  asos_not_12 = []
  for i, row in enumerate(asos):
    row_fst = row[0]

    for idx, r in enumerate(row):
      if idx != 0:
        row_fst = np.concatenate([row_fst, r])

    row_fst = row_fst.reshape(-1).tolist()

    if len(row_fst) != 192:
      asos_not_12.append(i)
    
    else:
      asos_np.append(row_fst)

  asos_np = np.array(asos_np)

  station_id = list(set(asos_dd.STATION_ID))
  station_id.sort()

  df = pd.DataFrame()

  for idx, id in enumerate(station_id):
    if not idx in asos_not_12:
      df = pd.concat([df, asos_dd[asos_dd.STATION_ID == id].head(1)[['STATION_ID', 'STATION_NAME']]], axis=0)

  df = np.array(df)
  df = np.concatenate([df, asos_np], axis=1)

  asos_columns = ['STATION_ID', 'STATION_NAME']
  [asos_columns.extend([f'CSS_AVG_{month:02}', f'MIN_CSS_AVG_{month:02}', f'MAX_CSS_AVG_{month:02}', f'MAX_IWS_DEG_{month:02}',
                        f'MAX_WS_MS_{month:02}', f'MAX_WS_DEG_{month:02}', f'WS_AVG_MS_{month:02}', f'MAX_WO_DEG_{month:02}',
                        f'CSS_DEW_AVG_{month:02}', f'MIN_REL_HUMID_{month:02}', f'REL_HUMID_AVG_{month:02}', f'VAPOR_AVG_{month:02}',
                        f'VAPOR_LOCAL_AVG_{month:02}', f'PDOS_HR_{month:02}', f'CSS_GRD_AVG_{month:02}', f'MIN_CSS_GRD_{month:02}']) for month in range(1, 13)]

  asos_df = pd.DataFrame(data=df, columns=asos_columns)

  station = pd.read_csv(str(path / 'station' / 'station.csv'))

  aws_station_list = []
  cnt = 0
  for row in aws_df.itertuples():

    aws_station_list.append(station[station.STATION_ID == row[1]].tail(1)[['LATITUDE', 'LONGITUDE']].to_numpy().reshape(-1))

  aws_station_list = np.array(aws_station_list)
  print(aws_station_list.shape)
  aws_station_list = pd.DataFrame(data=aws_station_list, columns=['LATITUDE', 'LONGITUDE'])

  asos_station_list = []
  cnt = 0
  for row in asos_df.itertuples():
    
    asos_station_list.append(station[station.STATION_ID == row[1]].tail(1)[['LATITUDE', 'LONGITUDE']].to_numpy().reshape(-1))

  asos_station_list = np.array(asos_station_list)
  print(asos_station_list.shape)
  asos_station_list = pd.DataFrame(data=asos_station_list, columns=['LATITUDE', 'LONGITUDE'])

  aws_df = pd.concat([aws_df, aws_station_list], axis=1)
  asos_df = pd.concat([asos_df, asos_station_list], axis=1)

def extract_lat_log(local_df):
  p = re.compile(r'[a-zA-Z(())]')

  lat_log = []
  for row in local_df.itertuples():

    lat, log = map(float, re.sub(p, '', row[1]).strip().split(' '))

    lat_log.append([lat, log])
  
  return np.array(lat_log)


# sorce => 수정 필요
def calculate_distance(df):
  p = re.compile(r'[a-zA-Z(())]')

  cnt = 0

  min_id_list = []
  for row in df.itertuples():

    cnt += 1

    if cnt % 500000 == 0:
      print(row) 

    lat, log = map(float, re.sub(p, '', row[1]).strip().split(' '))

    min_d = 1000000000
    min_id = -1

    for id in station.itertuples():
      tmp_d = (lat - id[4])**2 + (log - id[5])**2
      if min_d > tmp_d:
        min_d = tmp_d
        min_id = id[1]

    min_id_list.append(min_id)  
  return min_id_list

## apply_kmeans_station

In [ ]:
def make_kmeans_station(station, path):
    # 지점 클러스터링
  X = np.array(station[['STATION_ID','LATITUDE', 'LONGITUDE']])

  kmeans = KMeans(n_clusters=95, random_state=41).fit(X[:, 1:3])

  X_label = kmeans.labels_[:, np.newaxis]

  label = np.concatenate([X, X_label], axis=1)

  label_df = pd.DataFrame(data=label, columns=['STATION_ID', 'LATITUDE', 'LONGITUDE', 'LABEL'])
  
  label_df[['STATION_ID','LABEL']] = label_df[['STATION_ID', 'LABEL']].astype(int) 
  sns.scatterplot(data=label_df, x='LATITUDE', y='LONGITUDE', hue='LABEL')

  label_dict = {}

  for row in label_df.itertuples():
    label_dict[row[4]] = row[1]

  with open(str(path / 'models' / 'kmeans'), 'wb') as f:
    pickle.dump(kmeans, f, pickle.HIGHEST_PROTOCOL)

  with open(str(path / 'models' / 'kmeans_label'), 'wb') as f:
    pickle.dump(label, f, pickle.HIGHEST_PROTOCOL)
  
  return kmeans, label_dict


def apply_kmeans_station(kmeans, label_dict, find_sec_path, save_sec_path):
  gis_csv_list = glob(str(path / find_sec_path / '*.csv'))
  gis_csv_list.sort()

  for gis in gis_csv_list:

    gis_df = pd.read_csv(gis)
    if 'STATION_ID' in gis_df.columns:
      gis_df = gis_df.drop(['STATION_ID'], axis=1)
    
    pred = gis_df[['LATITUDE', 'LONGITUDE']]
    pred_np = pred.to_numpy()

    pred_lat = pred_np[:, 0]
    pred_log = pred_np[:, 1]

    pred_cls_label = kmeans.predict(pred_np)

    pred_id = pd.DataFrame(data=[label_dict[row] for row in pred_cls_label], columns=['STATION_ID'])

    gis_df = pd.concat([gis_df, pred_id], axis=1)

    gis_df.to_csv(str(path / save_sec_path / gis.split('/')[-1]), index=False)

  sns.scatterplot(data=gis_df, x='LATITUDE', y='LONGITUDE', hue='STATION_ID', palette="Paired")

## preprocess_asos

In [ ]:
def plus_asos_aws(asos,aws, station):
  station = station.sort_values(['DATE'], ascending=False)
  total_min_id = []
  for row in asos.itertuples():

    min_d = 100**6
    min_id = -1

    for r in  aws.itertuples():
      present_d = (r[87] - row[195])**2 + (r[88] - row[196])**2
      if present_d < min_d:
        min_id = r[1]
        min_d = present_d

    total_min_id.append(min_id)

  asos['AWS_STATION_ID'] = total_min_id

  df_plus = pd.DataFrame()

  for id in total_min_id:
    df_plus = pd.concat([df_plus, aws[aws.STATION_ID == id].head(1)], axis=0)

  df_plus = df_plus[['RF_AVG_01', 'RF_AVG_02', 'RF_AVG_03', 'RF_AVG_04',
                    'RF_AVG_05', 'RF_AVG_06', 'RF_AVG_07', 'RF_AVG_08',                  
                    'RF_AVG_09', 'RF_AVG_10', 'RF_AVG_11', 'RF_AVG_12']]

  asos = asos.reset_index(drop=True)
  df_plus = df_plus.reset_index(drop=True)

  asos = pd.concat([asos, df_plus], axis=1)

  print(f'len(total_min_id): {len(total_min_id)}')
  print(f'len(asos): {len(asos)}')
  print(f'len(df_plus): {len(df_plus)}')

  asos

  asos.to_csv(str(path / 'weather' / 'asos_df'), index=False)


## concat_gis_weather

In [ ]:
def concat_gis_weather(station, df_local_bug, df_local_safe):
  station = station.sort_values('DATE', ascending=False)

  df = pd.DataFrame()

  for row in df_local_bug.itertuples():
    tmp_df = station[station.STATION_ID == row[-1]].head(1)
    df = pd.concat([df, tmp_df], axis=1)

  df_jeju_bug = pd.concat([df_jeju_bug, df], axis=1)
